## Importer le modèle

In [1]:
from keras.models import load_model
import numpy as np
model = load_model('../modèles_assez_bons/05.06.2020/model3.h5')
# summarize model.
model.summary()

Using TensorFlow backend.


Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 88, 88, 32)        896       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 44, 44, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 42, 42, 16)        4624      
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 21, 21, 16)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 19, 19, 16)        2320      
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 9, 9, 16)          0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 9, 9, 16)         

## Importer l'image

In [2]:
import rasterio
image_path = '../Données/S2A_MSIL2A_20190227T030651_N0211_R075_T48PYU_20190227T083207/S2A_MSIL2A_20190227T030651_N0211_R075_T48PYU_20190227T083207.SAFE/GRANULE/L2A_T48PYU_A019234_20190227T031435/IMG_DATA/R10m/T48PYU_20190227T030651_'

In [3]:
IMAGE_PIXEL = 90
IMAGE_WIDTH = 122
BAND_WIDTH = 3

In [4]:
def reshape(image):
    reshaped_image = []
    for i in range(len(image[0])):
        reshaped_row = []
        for j in range(len(image[0][0])):
            reshaped_cell = []
            #rajouter / 65535
            reshaped_cell.append(image[0][i][j] / 65535)
            reshaped_cell.append(image[1][i][j] / 65535)
            reshaped_cell.append(image[2][i][j] / 65535)
            reshaped_row.append(reshaped_cell)
        reshaped_image.append(reshaped_row)
    return reshaped_image

In [5]:
def split_band(band):
    """Split une bande de Sentinel-2 en bande plus petite
    2 916 images de 20x20"""
    result = []
    my_band = band.read(1)
    for x in range(IMAGE_WIDTH):
        columns = []
        for y in range(IMAGE_WIDTH):
            lines = []
            for i in range(IMAGE_PIXEL):
                cells = []
                for j in range(IMAGE_PIXEL):
                    cells.append(my_band[i + IMAGE_PIXEL * x][j + IMAGE_PIXEL * y])
                lines.append(cells)
            columns.append(lines)
        result.append(columns)
    return result

In [6]:
def predict(image_path, output_path, model):
    """Créée des images plus petites pouvant être mieux traitées par le réseau de neurones"""
    green = rasterio.open(image_path+'B03_10m.jp2', driver='JP2OpenJPEG') #green
    red = rasterio.open(image_path+'B04_10m.jp2', driver='JP2OpenJPEG') #red
    infra_red = rasterio.open(image_path+'B08_10m.jp2', driver='JP2OpenJPEG') #near impact
    smaller_green = split_band(green)
    print("Fin bande verte")
    smaller_red = split_band(red)
    print("Fin bande rouge")
    smaller_infra_red = split_band(infra_red)
    print("Fin bande infra rouge")
    prediction = []
    prediction_len = set()
    for x in range(IMAGE_WIDTH):
        row_prediction = []
        for y in range(IMAGE_WIDTH):
            smaller_image = []
            smaller_image.append(smaller_infra_red[x][y])
            smaller_image.append(smaller_red[x][y])
            smaller_image.append(smaller_green[x][y])
            value = model.predict(np.expand_dims(reshape(smaller_image), axis=0))
            prediction_len.add(value[0][0] * 65535)
            for i in range(IMAGE_PIXEL):
                row_prediction.append(value[0][0] * 65535)
                
        for i in range(IMAGE_PIXEL):
            prediction.append(np.array(row_prediction).astype('uint16'))
    output = rasterio.open(output_path + '_prediction.tiff', 'w', driver='Gtiff', 
                          width=red.width, height=red.height, count=1, crs=red.crs, 
                          transform=red.transform, 
                          dtype=red.dtypes[0])
    output.write(prediction, 1)
    output.close()
    print(prediction_len)
    print(len(prediction_len))
    print("Le fichier de prédiction a été créé.")

In [7]:
predict(image_path, "output", model)

Fin bande verte
Fin bande rouge
Fin bande infra rouge
{0.04303058141488236, 0.012803774531704448, 0.2572685860536694, 2.600595130770671, 0.01815717900939262, 0.07519829266925626, 0.06723731231602414, 2.050521317632956, 0.5564527836986599, 2.687214855104685, 2.793233926367975, 1.5275739988737769, 1.3961079472846905, 2.2508241999275924, 13.94388474887819, 15.260739159057266, 3.228366704315704, 7.249113331217245e-18, 7.849770466568983e-18, 3.472640931031492, 8.569545582768767e-18, 3.63778142414958, 21.95313143209205, 23.855514818133088, 1.0166072447786325e-17, 1.3479723962204988e-18, 26.19809420022648, 1.1495743338626853e-17, 1.1623845191022745e-17, 1.1808830558976486e-17, 5.787842648605874, 6.8478407219663495, 6.255074422151665, 1.3743759293533463e-17, 29.593212605832377, 6.053966174185916, 36.20139004604425, 1.6009040725911593e-17, 29.94436920431326, 30.95964111154899, 40.06705371604767, 8.944177523590042, 42.63073529989924, 8.767808640404837, 3.687909462022351e-18, 3.6080356393288026e-

In [8]:
output = rasterio.open('output_prediction.tiff', 'r', driver='Gtiff')

In [9]:
print(output.count)
print(output.width)
print(output.height)
print(output.read())
set_output = set()
for i in output.read(1):
    for j in i:
        set_output.add(j)
print(set_output)
print(len(set_output))

1
10980
10980
[[[    0     0     0 ... 65534 65534 65534]
  [    0     0     0 ... 65534 65534 65534]
  [    0     0     0 ... 65534 65534 65534]
  ...
  [65140 65140 65140 ...     0     0     0]
  [65140 65140 65140 ...     0     0     0]
  [65140 65140 65140 ...     0     0     0]]]
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 57364, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 57378, 38, 49190, 40, 41, 42, 39, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 57399, 57, 58, 57403, 60, 61, 62, 59, 64, 65, 66, 49216, 68, 69, 70, 71, 63, 73, 8266, 72, 49228, 77, 78, 79, 76, 81, 24657, 83, 24660, 85, 86, 87, 88, 89, 32858, 90, 91, 92, 94, 49246, 95, 97, 98, 99, 100, 101, 20, 103, 57443, 105, 106, 107, 102, 57453, 110, 57455, 112, 111, 114, 115, 116, 16501, 118, 119, 120, 117, 122, 49271, 124, 125, 16507, 127, 128, 129, 130, 131, 49281, 133, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 57489, 148, 149, 150, 151, 41112, 153, 154, 1